# Numerical Methods - Biswa Datta

## 3.2.4 Catastrophic Cancellation

When computers perform arithmetic operations such as addition, subtraction etc. It does not perform them the way humans do. Instead they perform fixed-precision arithmetic using only 64 bits, i.e. in order to hold results they use finite number of decimal places, this means numbers get rounded to the nearest finite number. Not paying attention to the possible problems arising from this has real [consequences](http://www-users.math.umn.edu/~arnold/disasters/).

One of the problems seen when working with limited precision is known as Catastrophic Cancellation. In order to explain this concept I'll use *decimal* - a python package which offers arbitrary-precision arithmetic.

Say I want to compute $f(0.01)$ (as 64 digits are hard to work with, let's work with $5$ significant digits) where $f$ is,

$$f(x) = e^x - 1 - x$$

In 5 precision arthemetic, we can't hold the whole of $e$, so we round **up** $2.71828182845904$ to $2.7183$ as $28>25$. 


$$ f_5(0.01) = (2.7183)^{0.01}-1-0.01$$

Here $(2.7183)^{0.01} = 1.0100502346051552$, but again remember only 5 significant digits allowed! As $502>500$ we go with $1.0101$.

At this point note that the rounding that we did didnt introduce any significant error - just $\approx 10^{-5}$, which is expected considering that we are using only 5 significant digits.

$$\epsilon = |1.0100502346051552 - 1.0101 | = 4.976539484480291 \times 10^{-5}$$
$$\eta = \frac{4.976539484480291 \times 10^{-5}}{1.0100502346051552} = 4.927021759888705 \times 10^{-5}$$

Now comes the "cancellation",

$$ f_5(0.01) = 1.0101-1-0.01 = 0.0001$$

In [1]:
import math
import decimal
from decimal import *
from math import e

precision = 5
getcontext().prec = precision

val = 0.0100
decX = Decimal(val)
decE = Decimal(e)+Decimal(0)
print(decE)

def fun(x):
    print("i.e. e raised to x equals")
    print(decE**x)
    return(decE**x-Decimal(1)-x)

print("\nRemoving 1.0 and 0.01 we have \nf(0.01) = "+'{0:.4f}'.format(fun(decX)))

2.7183
i.e. e raised to x equals
1.0101

Removing 1.0 and 0.01 we have 
f(0.01) = 0.0001


So this looks OK right, this should be the answer, not an exact value but should atleast be around 5 decimal places close to the answer.

But no, the actual answer is $0.00005$ (upto 5 decimal places).

In [3]:
# Moving on to use all 53 bits of precision offered by python float
real = (e**val)-1-val
print("\nThe real answer is")
print('{0:.10f}'.format(real))

print("\nwhich we get this by subtracting x and 1 from e^x,")
print(e**val)
print(e**val-val)
print('{0:.10f}'.format(real))


The real answer is
0.0000501671

which we get this by subtracting x and 1 from e^x,
1.010050167084168
1.000050167084168
0.0000501671


Notice how the relative error is now $0.9933$ which is nearly 100,000 times worse! We get a percentage error of $99.33\%$

$$\epsilon = |0.0000501671 - 0.0001| = 4.98329 \times 10^{-5}$$
$$\eta = \frac{4.98329 \times 10^{-5}}{0.0000501671} = 0.9933382635233051$$

This error was introduced when we "rounded to nearest" and starting working with $1.0101$ instead of $1.01005$, note even if had rounded the other way ($1.0100$) we would still have around this same error. The error is due to us working with finite precision.

But the error didn't really matter when value we cared about was around $1$, the reason the [relative error](https://en.wikipedia.org/wiki/Approximation_error#Formal_Definition) shot up was because the "actual" value became close to zero when we subtracted the close numbers. And **the closer you are to zero the more important these significant digits that we lost become.**

Now the wikipedia article on [Catastrophic Cancellation](https://en.wikipedia.org/wiki/Loss_of_significance) should make sense, note how the absolute error is still around $10^{-5}$, not much change there. It is the relative error which increased "substantially" and the key point is that relative error for any value near zero will be large.

Now you might be like, its just the relative error that so high, the absolute error is still really small. Its not that bad.

Here its important to understand the relevance of relative error, an absolute error of $10^{-5}m$ can be neglected when measuring something like the height of a room, but the *same* absolute error in measuring the diameter of a human hair (typically 17 to 181 μm i.e. $10^{-6}m$) would mean that you're approximation is really wrong. 

This is why relative error matters, we need to be able see the absolute error in the context of the actual value. Now its clear why small errors caused by rounding off suddenly start to matter when the value in question is also very small.

A solution to this problem is to use alternative ways to compute what we want, in a way that doesnt involve catastrophic cancellation.

In our case we know,

$$e^x = 1+x+\frac{x^2}{2!} + \frac{x^3}{3!} + \cdots$$

So we can find $f(x)$ by adding positive numbers, avoiding subtraction (works only when x is positive)

$$f(x) = e^x - 1 - x = \frac{x^2}{2!} + \frac{x^3}{3!} + \cdots$$


In [4]:
# Alternatively using the power series
temp = Decimal(0.00000)

k = 3
for i in range(2,k+2):
    j = Decimal(i)
    temp = temp + decX**i/math.factorial(j)
    # A single term from that series gives the right result
    print("\n\nThe sum of first "+str(i-1)+" terms are,")
    print(temp)




The sum of first 1 terms are,
0.00005000


The sum of first 2 terms are,
0.000050167


The sum of first 3 terms are,
0.000050167


### A different example

Finally I'd like to mention a popular example - finding the roots to a quadratic equation. 

$$ax^2+bx+c = 0$$

When $\sqrt{b^2-4ac} \approx b$, then one root is likely to be wrong if we compute via the formula.

$$\frac{-b+\sqrt{b^2-4ac}}{2a}$$

and its not like this is unlikely, a small $c$ or $a$ will cause $4ac \approx 0$.

In which case the smart way is to find one root by subtraction,


$$\frac{-b-\sqrt{b^2-4ac}}{2a}$$

And the second root should be,

$$= \frac{\frac{c}{a}}{\frac{-b-\sqrt{b^2-4ac}}{2a}}$$

In the example below we see $99.975\%$ percentage error!

In [2]:
a = Decimal(10**-3)
b = Decimal(2)
c = Decimal(10**-3)

# working with 5 significant digits
print('{0:.5f}'.format(a)+"x^2 + "+'{0:.5f}'.format(b)+"x + "+'{0:.5f}'.format(a))

estiRoot1 = (-b+(b**2-4*a*c).sqrt())/(2*a)
print("Root one is "+'{0:.5f}'.format(estiRoot1))

estiRoot2 = (-b-(b**2-4*a*c).sqrt())/(2*a)
print("Root two is "+'{0:.5f}'.format(estiRoot2))

print("But the actual root one is "+'{0:.5f}'.format((c/a)/estiRoot2))

# we can't hold 10^-6, we round it to 0
print("The error is because of how sqrt(b^2-4ac) approx b, i.e. = "+str((b**2-4*a*c).sqrt()))

error = abs(estiRoot2 - estiRoot1)/abs(estiRoot2)
print("The error in percentage is "+str(100*error))

0.00100x^2 + 2.00000x + 0.00100
Root one is 0.00000
Root two is -2000.00000
But the actual root one is -0.00050
The error is because of how sqrt(b^2-4ac) approx b, i.e. = 2.00
The error in percentage is 100


## Sample Problems

Prof. Ramakalyan had asked the following question in the End-Term Test for PG students,

**1. (b) [4 marks] Show how to rewrite the following expressions to avoid cancellation for the indicated arguments:**


$$(i) \ \  \sqrt{x + 1} − 1 \ \ , \ \ x ≈ 0$$

$$(ii) \ \  \sin(x) − \sin(y) \ \  ,\ \   x ≈ y$$
     
$$(iii) \ \  x^2 − y^2 \ \   ,  \ \ x ≈ y $$
     
$$(iv) \ \   \frac{(1 − \cos(x))}{\sin(x)} \ \  , \ \   x ≈ 0$$
     

Coming to (i), I thought of either using taylors series or the [Generalized binomial theorem](https://en.wikipedia.org/wiki/Binomial_theorem#Newton.27s_generalized_binomial) because in our case $n$ is not a nonnegative integer.

But [it turns out](https://math.stackexchange.com/questions/905361/binomial-expansion-taylor-series-and-power-series-connection) they both give the same infinite series. Makes sense due to how the power series for a function is [unique](https://math.stackexchange.com/questions/83951/uniqueness-of-power-series).

So let's write the function as a power series,

$$g(x) = {\displaystyle \sum _{n=0}^{\infty }{\frac {g^{(n)}(a)}{n!}}\,(x-a)^{n}} $$

In our case,

$$f(x) = \sqrt{x + 1} = (x+1)^{\frac{1}{2}}$$

$$f^1(x) = \frac{1}{2(x+1)^{\frac{1}{2}}}$$

Similarly by repeatedly applying power rule,

$$f^n(x) = \frac{d^nf(x)}{dx^n} = \frac{\prod_{k=0}^n (1-2k)}{2^n \times (x+1)^{\frac{2n-1}{2}}}$$

Lets take the special case of taylors series when $a=0$, i.e. Maclaurin series.

$$f^n(0) = \frac{\prod_{k=0}^n (1-2k)}{2^n}$$

So we can write 

$$f(x) = \sqrt{x + 1} =  \sum_{n=0}^\infty \frac{f^{(n)}(0)}{n!}x^{n} = \sum_{n=0}^\infty \frac{\prod_{k=0}^n (1-2k)}{2^n n!}x^n$$

In [15]:
# n^th differentiation of (1+x)^{1/2}

def foo(n,x):
    num = 1
    for k in range(0,n):
        num = num * (1-2*k)
    print("The numerator is "+str(num))
    
    den = 2**n*pow((x+1),(2*n-1)/2)
    print("The constant denominator is "+str(2**n))
    return(num/den)

print(foo(11,6))

The numerator is 654729075
The constant denominator is 2048
0.00042776205246675527


I went and verified the formula for $n^{th}$ differential of the function using Wolfgramalpha (https://www.wolframalpha.com/input/?i=11th+differentiation+of+(x%2B1)%5E%7B1%2F2%7D+where+x+%3D+6)

![Result](http://i.imgur.com/BMI7u2I.png)

So, as we care about $\sqrt{x + 1}$ at $x \approx 0$, we can truncate the power series after a few terms. As further terms in the power series will contribute less and less number of significant digits.

Plugging in the values we find the first few terms in the series are,

$$\sqrt{x + 1} = \sum_{n=0}^\infty \frac{\prod_{k=0}^n (1-2k)}{2^n n!}x^n$$
$$ = \frac{1 \times x^0}{2^0 0!} +  \frac{-1 \times x^1}{2^1 1!} + \frac{3 \times x^2}{2^2 2!} + \frac{-15 \times x^3}{2^3 3!} + \cdots$$
$$= 1+  \frac{-x}{2} + \frac{3 x^2}{8} + \frac{-5 x^3}{16} + \mathcal{O}(x^4)$$

The last term just the truncation upto 4 terms we did would result in an error of order $x^4$, read more about the big O notation [here](https://math.stackexchange.com/a/109572/308392)

Now finally coming to our actual answer,

$\sqrt{x + 1} − 1$ can be written as,

$$1-1 + \frac{-x}{2} + \frac{3 x^2}{8} + \frac{-5 x^3}{16} = -\frac{x}{2} + \frac{3 x^2}{8} - \frac{5 x^3}{16}$$

Let us see how this way of computing fares against the naive way of computing $\sqrt{x + 1} − 1$.